# Kryptologie Lab - Übung 07 - RSA Key Gen

## RSA

In [1]:
import random
import math

In [2]:
class RSA:
    small_primes = [1,7,11,13,17, 19,23,29]

    def eea(self, a,b):
        k = 0
        r = [a, b]
        s = [1, 0]
        t = [0, 1]
        while r[k+1] != 0:
            k += 1
            q_k = r[k-1] // r[k]
            r.append(r[k-1]-q_k*r[k])
            s.append(s[k-1]-q_k*s[k])
            t.append(t[k-1]-q_k*t[k])
        return (r[k], s[k], t[k])

    # x^m mod n
    def potModN(self, x, m, n):
        y = 1
        b = m
        while b != 0:
            if (b & 0x1) == 1:
                y = (y * x) % n
            x = (x*x) % n
            b = b >> 1
        return y

    # Nichtdeterministischer Primzahltest
    # Keine Fals Negatives, 25% false positives
    # returns true if prime
    def millerRabin(self, n):
        # determine k and m
        temp = n-1
        k = 0
        while (temp % 2) == 0:
            temp //= 2
            k += 1
        m = temp
        # -> n-1 = 2^k*m
        # determine a
        a = random.randrange(2,n)
        b = self.potModN(a, m, n)
        if b == 1:
            return True
        for i in range(1, k+1):
            if b == n-1:
                return True
            else:
                b = self.potModN(b, 2, n)
        return False
        

    # execute miller rabin 8 times to cancel out false positives
    def testPrim(self, n):
        success = False
        for i in range(8):
            success = self.millerRabin(n)
            if success: return True
        return False

    def genPrim(self, max_z = 10**130):
        z = random.randint(1000, max_z)
        hit = False
        i = 1
        while not hit:
            number = 30 * z + self.small_primes[i]
            hit = self.testPrim(number)
            i += 1
            if i == 8:
                i = 0
                z += 1
        return number
    
    def genKeyPair(self):
        # generate primes
        p = self.genPrim()
        q = self.genPrim()
        # p and q shouldn't be too close together
        while min([p,q]) * 1.2 > max([p,q]):
            q = self.genPrim()
        # gen e (teilerfremd zu phi_n)
        n = p * q
        phi_n = (p-1) * (q-1)
        hit = False
        while not hit:
            e = random.randint(1000, 10**180)
            ggt, d, _ = self.eea(e, phi_n)
            hit = ggt == 1
        if d < 0:
            d = phi_n + d
        return ((e,n), (d, n))


    def encrypt(self, x, public):
        e, n = public
        return self.potModN(x,e,n)

    def decrypt(self, x, private):
        d, n = private
        return self.potModN(x,d,n)
        

In [3]:
rsa = RSA()
(public, private) = rsa.genKeyPair()
print("PUBLIC")
print("e - ", (int(math.log10(public[0]))+1), " decimal digits")
print("n - ", (int(math.log10(public[1]))+1), " decimal digits")
print("PRIVATE")
print("d - ", (int(math.log10(private[0]))+1), " decimal digits")
print("n - ", (int(math.log10(private[1]))+1), " decimal digits")

print("n: ", public[1])

PUBLIC
e -  180  decimal digits
n -  263  decimal digits
PRIVATE
d -  263  decimal digits
n -  263  decimal digits
n:  55346405440131148053752343873295115458743125518065208682619386347600684501568320557694362406880015078582202506148992459629308542496074146804708358509720884301008477799130006474685548308665662975008395629295687734875601798404399128821705486328981745696118401859431


In [4]:
input = 999999999
encrypted = rsa.encrypt(input, public)
print(encrypted)
output = rsa.decrypt(encrypted, private)
assert output == input

4451164950347409967829307197831994633003612069969558407593906492914055141390826728822084602025286357644708600155215205189437888144470906979671211802811146335169964692046611262211762957725797183114265052397189936604118693583188627578042011185559093983794913934579


## Differenz der Quadrate

In [5]:
# Idee: Suche N = u^2 - w^2. Daraus folgt N = (u-w)(u+w)
def diffOfQuads(n):
    u = math.ceil(math.sqrt(n))
    while not math.sqrt(u*u-n).is_integer():
        u += 1
        if u > n:
            raise Exception("Couldn't find factorization")
    w = math.sqrt(u*u - n)
    return (int(u+w), int(u-w))

In [6]:
n = 98939*85081
diffOfQuads(n)

(98939, 85081)